In [1]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)
import os

import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from callbacks import all_callbacks

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
import tensorflow.compat.v1 as tf1

2023-07-22 03:29:43.293583: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-22 03:29:43.388232: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Fetch the cardio dataset

In [2]:
df = pd.read_csv('./dataset.csv', sep = ';')
print (np.shape(df))

(2126, 22)


In [3]:
X = df.drop('Y', axis = 1).values
y = df.Y

In [4]:
le = LabelEncoder()
y = le.fit_transform(y)
print(le.classes_)
y = to_categorical(y, 3)

[1 2 3]


In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [6]:
ls=np.argmax(Y_test,axis=1)+1

In [7]:
print("Shape of X_train: ",X_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_train: ",Y_train.shape)
print("Shape of y_test",Y_test.shape)

Shape of X_train:  (1488, 21)
Shape of X_test:  (638, 21)
Shape of y_train:  (1488, 3)
Shape of y_test (638, 3)


In [ ]:
X_test

In [67]:
norm = 2**4
sc = MinMaxScaler(feature_range=(0,0.9))
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
# for i in range(0,len(X_train)):
#    X_train[i] = [int(x*norm)/norm for x in X_train[i]]
# for i in range(0,len(X_test)):
#    X_test[i] = [int(x*norm)/norm for x in X_test[i]]

In [18]:
import pickle
from joblib import load
loaded_model = load('./Cardio.MLP_clf.joblib')

/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [139]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

clf = MLPClassifier(beta_1=0.004044058262057914, beta_2=0.2692099545241596,
              epsilon=0.4100816459563625, hidden_layer_sizes=3, max_iter=150,
              momentum=0.8221177331942455, nesterovs_momentum=False,
              solver='lbfgs', validation_fraction=0.511318982546456,alpha=0.0001).fit(X_train, Y_train)
clf.score(X_test, Y_test)

/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.8887147335423198

In [145]:
w1=loaded_model.coefs_[0]
b1=loaded_model.intercepts_[0]
w2=loaded_model.coefs_[1]
b2=loaded_model.intercepts_[1]

In [146]:
wb1=[]
wb1.append(w1)
wb1.append(b1)

wb2=[]
wb2.append(w2)
wb2.append(b2)

In [147]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

import tensorflow as tf
import os
import pandas as pd
import seaborn as sb

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from callbacks import all_callbacks
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu,quantized_po2
import tensorflow.compat.v1 as tf1
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from qkeras.utils import model_save_quantized_weights
from sklearn.preprocessing import MinMaxScaler

In [148]:
model=Sequential()

model.add(QDense(3, input_shape=(21,), name='fc1', kernel_quantizer=quantized_bits(8,0,alpha=1,use_stochastic_rounding=True),bias_quantizer=quantized_bits(8,0,alpha=1),
                kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)   ))
model.add(QActivation(activation=quantized_relu(8,0,use_stochastic_rounding=False), name='relu1'))
model.add(QDense(3, name='output',
                kernel_quantizer=quantized_bits(8,0,alpha=1,use_stochastic_rounding=True), bias_quantizer=quantized_bits(8,0,alpha=1),
                kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001 ) ))
model.add(Activation(activation='softmax', name='softmax'))
model.layers[0].set_weights(wb1)
model.layers[2].set_weights(wb2)

In [149]:
adam = Adam(lr=0.003)

model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
callbacks= all_callbacks( outputDir = 'cardio_classification_prune')

model.fit(X_train, Y_train, batch_size=1,
          epochs=5,validation_split=0.2, verbose=1, shuffle=True,
          callbacks = callbacks.callbacks);
model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
model_save_quantized_weights(model, "test_weights")

Epoch 1/5
1190/1190 [==============================] - 4s 3ms/step - loss: 0.4239 - accuracy: 0.8412 - val_loss: 0.3521 - val_accuracy: 0.8691 - lr: 0.0010
Epoch 2/5
1190/1190 [==============================] - 3s 2ms/step - loss: 0.3961 - accuracy: 0.8546 - val_loss: 0.3442 - val_accuracy: 0.8792 - lr: 0.0010
Epoch 3/5
1190/1190 [==============================] - 3s 3ms/step - loss: 0.3860 - accuracy: 0.8580 - val_loss: 0.3379 - val_accuracy: 0.8826 - lr: 0.0010
Epoch 4/5
1190/1190 [==============================] - 3s 2ms/step - loss: 0.3793 - accuracy: 0.8580 - val_loss: 0.3367 - val_accuracy: 0.8893 - lr: 0.0010
Epoch 5/5
1190/1190 [==============================] - 3s 3ms/step - loss: 0.3754 - accuracy: 0.8555 - val_loss: 0.3290 - val_accuracy: 0.8758 - lr: 0.0010
... quantizing model


{'fc1': {'weights': [array([[-0.34375  ,  0.8671875, -0.09375  ],
          [ 0.9921875, -1.       , -0.859375 ],
          [-0.265625 ,  0.75     ,  0.203125 ],
          [ 0.9921875, -0.734375 ,  0.40625  ],
          [ 0.40625  ,  0.1015625,  0.28125  ],
          [-0.203125 ,  0.1328125,  0.1484375],
          [-0.984375 ,  0.9765625,  0.9765625],
          [-0.7578125,  0.8125   ,  0.5      ],
          [ 0.3671875, -0.703125 ,  0.671875 ],
          [-0.6875   ,  0.9140625, -0.671875 ],
          [ 0.7109375, -0.1640625, -0.09375  ],
          [ 0.2890625,  0.1875   , -0.015625 ],
          [ 0.1171875, -0.234375 , -0.453125 ],
          [-0.0546875,  0.296875 ,  0.0625   ],
          [-0.03125  , -0.1640625,  0.0390625],
          [ 0.03125  , -0.1640625,  0.375    ],
          [ 0.171875 , -0.1171875, -0.5390625],
          [ 0.4296875,  0.0078125, -0.53125  ],
          [ 0.       ,  0.046875 , -0.96875  ],
          [-0.984375 ,  0.671875 ,  0.96875  ],
          [ 0.0234375,

In [151]:
model.save("CARDIO_MODEL")
#model = tf.keras.models.load_model("CARDIO_MODEL")
accuracy=model.evaluate(X_test,Y_test)

INFO:tensorflow:Assets written to: CARDIO_MODEL/assets


INFO:tensorflow:Assets written to: CARDIO_MODEL/assets


20/20 [==============================] - 0s 2ms/step - loss: 0.3878 - accuracy: 0.8824


In [157]:
import estimate as es
from importlib import reload
reload(es)

<module 'estimate' from '/home/edge/Desktop/argykokk/hls4ml-tutorial/networks/cardio/estimate.py'>

In [168]:
param=2
reuse=100

In [169]:
%%time
#input_num, neurons_num, layer_id, model
luts, ffs = es.estimate(21,3,0,model,reuse,param)
luts, ffs = es.estimate(3,3,2,model,reuse,param)

Mul ins = 32 and Max muls = 1 and Saved muls = 31 Reuse factor = 100
Muxes LUTS: 403
DSPs: 1
LUT cost2= 377 bias acc= 42 mult acc826
LUTs prediction: 1245
Initial muls: 63 Real muls: 62 Initial neurons: 3 Tuned neurons: 3
FFs prediction: 1959
Mul ins = 1 and Max muls = 1 and Saved muls = 0 Reuse factor = 100
Muxes LUTS: 0
DSPs: 1
LUT cost2= 128 bias acc= 42 mult acc84
LUTs prediction: 254
Initial muls: 9 Real muls: 9 Initial neurons: 3 Tuned neurons: 3
FFs prediction: 771
CPU times: user 12.5 ms, sys: 67 µs, total: 12.6 ms
Wall time: 11.7 ms


/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [156]:
import hls4ml
import plotting

config = hls4ml.utils.config_from_keras_model(model, granularity='name')
config['LayerName']['softmax']['exp_table_t'] = 'ap_fixed<18,8>'
config['LayerName']['softmax']['inv_table_t'] = 'ap_fixed<18,4>'
print("-----------------------------------")
plotting.print_dict(config)
print("-----------------------------------")
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=config, output_dir='CARDIO/hls4ml_prj', part='xc7z007s-clg225-2'
)
hls_model.compile()

y_qkeras = model.predict(np.ascontiguousarray(X_test))
y_hls = hls_model.predict(np.ascontiguousarray(X_test))

None
Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: InputLayer, input shapes: [[None, 21]], output shape: [None, 21]
Layer name: fc1, layer type: QDense, input shapes: [[None, 21]], output shape: [None, 3]
Layer name: relu1, layer type: Activation, input shapes: [[None, 3]], output shape: [None, 3]
Layer name: output, layer type: QDense, input shapes: [[None, 3]], output shape: [None, 3]
Layer name: softmax, layer type: Softmax, input shapes: [[None, 3]], output shape: [None, 3]
-----------------------------------
Model
  Precision:         fixed<16,6>
  ReuseFactor:       1
  Strategy:          Latency
  BramFactor:        1000000000
  TraceOutput:       False
LayerName
  fc1_input
    Trace:           False
    Precision
      result:        fixed<16,6>
  fc1
    Trace:           False
    Precision
      result:        fixed<16,6>
      weight:        fixed<8,1>
      bias:          fixed<8,1>
      accum:         fixed<16,6>
    ReuseFactor:     1
  fc1_linear